### Introduction
----------------
This is a modified version of Tensorflow's Tutorial: [MNIST For ML Beginners](https://www.tensorflow.org/get_started/mnist/beginners).

### Data Description
----------------
* Training data is from **THE MNIST DATABASE of handwritten digits** (image + label = 60000 data): http://yann.lecun.com/exdb/mnist/
  * [train-images-idx3-ubyte.gz](http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz):  training set images (9912422 bytes) 
  * [train-labels-idx1-ubyte.gz](http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz):  training set labels (28881 bytes)
  

* Testing Data:
  * [t10k-images-idx3-ubyte.gz](http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz): test set images (1648877 bytes) 
  * [t10k-labels-idx1-ubyte.gz](http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz): test set labels (4542 bytes)

### References
----------------
Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based learning applied to document recognition." Proceedings of the IEEE, 86(11):2278-2324, November 1998. [[on-line version]](http://yann.lecun.com/exdb/publis/index.html#lecun-98)

### API References
----------------
The following links are the API references (ordered by the code):
* [gzip](https://docs.python.org/3/library/gzip.html)
* [numpy.frombuffer](https://docs.scipy.org/doc/numpy/reference/generated/numpy.frombuffer.html)
* [np.fromfile](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fromfile.html)
* [fseek](https://www.tutorialspoint.com/python/file_seek.htm)
* [fseek then fromfile](http://stackoverflow.com/questions/14245094/how-to-read-part-of-binary-file-with-numpy)
* [np.reshape](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html)
* [np.arange](https://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html)
* [numpy.setdiff1d](https://docs.scipy.org/doc/numpy/reference/generated/numpy.setdiff1d.html)
* [show reshape image from np.ndarray](http://stackoverflow.com/questions/2659312/how-do-i-convert-a-numpy-array-to-and-display-an-image)
* [numpy data types](https://docs.scipy.org/doc/numpy/user/basics.types.html)
* [numpy.random.choice](https://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.random.choice.html)
* [numpy.random.shuffle](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.shuffle.html)
* [numpy.mean](https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html)
* [tf.reduce_sum](https://www.tensorflow.org/api_docs/python/tf/reduce_sum)
* [tf.nn.softmax_cross_entropy_with_logits](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)

### Simple Gradient Descent Model (read data from original .gz file)
---
* I used python built-in module **time** to estimate elapsed time approximately:  
> ```python
> start = time.time()
> # ...
> # function to be estimated
> # ...
> end = time.time()
> print(end - start)
> 
> ```
* Loss Function (cross-entropy ):
> **$ H_{y'}(y) = -\sum_i y'_i \log(y_i) $**

In [1]:
import gzip, time
import numpy as np
import tensorflow as tf


def read_data(train_data_path, train_label_path,
              test_data_path, test_label_path):
    train_data, train_label = None, None
    with gzip.open(train_data_path, 'rb') as f: # train data
        train_data = np.reshape(np.frombuffer(f.read(), dtype=np.uint8, offset=16), (60000, -1))
    with gzip.open(train_label_path, 'rb') as f: # train label
        train_label = np.frombuffer(f.read(), dtype=np.uint8, offset=8)
    
    test_data, test_label = None, None
    with gzip.open(test_data_path, 'rb') as f: # test data
        test_data = np.reshape(np.frombuffer(f.read(), dtype=np.uint8, offset=16), (10000, -1))
    with gzip.open(test_label_path, 'rb') as f: # test label
        test_label = np.frombuffer(f.read(), dtype=np.uint8, offset=8)
    return train_data, train_label, test_data, gen_one_hot(np.array(test_label))


def gen_one_hot(labels, _class=10):
    one_hot_labels = np.zeros(shape=(np.array(labels).shape[0], _class), dtype=np.int)
    for index, label in enumerate(labels):
        one_hot_labels[index] = [1 if i==label else 0 for i in range(_class)]
    return one_hot_labels


def split_data(train_images, train_labels, num_img=60000, num_valid=5000, nb_classes=10):
    rand_list = np.random.choice(num_img, num_valid, replace=False)
    X_train = [train_images[i] for i in np.setdiff1d(np.arange(num_img), rand_list)]
    X_valid = [train_images[i] for i in rand_list]
    Y_train = [train_labels[i] for i in np.setdiff1d(np.arange(num_img), rand_list)]
    Y_valid = [train_labels[i] for i in rand_list]
    Y_train = np.array(gen_one_hot(np.array(Y_train), nb_classes))
    Y_valid = np.array(gen_one_hot(np.array(Y_valid), nb_classes))
    return np.array(X_train), np.array(X_valid), Y_train, Y_valid


# data preprocessing part
# ------------------------------------------------------
start = time.time()

data_start = time.time()

prefix = './data/MNIST_data/'
train_images, train_labels, test_imgs, test_labels = read_data(
    prefix+'train-images-idx3-ubyte.gz',
    prefix+'train-labels-idx1-ubyte.gz',
    prefix+'t10k-images-idx3-ubyte.gz',
    prefix+'t10k-labels-idx1-ubyte.gz')
X_train, X_valid, Y_train, Y_valid = split_data(train_images, train_labels, num_valid=0)

data_end = time.time()


# tensorflow calculation part
# ------------------------------------------------------
tf_start = time.time()

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=tf.matmul(x, W) + b, labels=y_)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy) # Gradient Descent
# train_step = tf.train.AdamOptimizer().minimize(cross_entropy) # Adam

epoch = 1000
rnd = [np.random.choice(X_train.shape[0], 100, replace=False) for i in range(epoch)]
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
train_start = time.time()
for i in range(epoch):
    batch_xs, batch_ys = np.array([X_train[r] for r in rnd[i]]) , np.array([Y_train[r] for r in rnd[i]])
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
train_end = time.time()
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Test Accuracy: ' + str(sess.run(accuracy, feed_dict={x: test_imgs, y_: test_labels})))

tf_end = time.time()

end = time.time()
print('Data Preprocessing Elapsed Time: ' + str(data_end-data_start))
print('TensorFlow         Elapsed Time: ' + str(tf_end-tf_start))
print('Training           Elapsed Time: ' + str(train_end-train_start))
print('Total              Elapsed Time: ' + str(end-start))

Test Accuracy: 0.9027
Data Preprocessing Elapsed Time: 1.59175705909729
TensorFlow         Elapsed Time: 2.4740309715270996
Training           Elapsed Time: 0.9172837734222412
Total              Elapsed Time: 4.065868854522705


### Simple Gradient Descent Model (read data from tensorflow mnist dataset)
---
* I used python built-in module **time** to estimate elapsed time approximately:  
> ```python
> start = time.time()
> # ...
> # function to be estimated
> # ...
> end = time.time()
> print(end - start)
> 
> ```
* Loss Function (cross-entropy ):
> **$ H_{y'}(y) = -\sum_i y'_i \log(y_i) $**

In [2]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


# data preprocessing part
# ------------------------------------------------------
start = time.time()

data_start = time.time()

mnist = input_data.read_data_sets("./data/MNIST_data/", one_hot=True)

data_end = time.time()


# tensorflow calculation part
# ------------------------------------------------------
tf_start = time.time()

x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=tf.matmul(x, W) + b, labels=y_)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy) # Gradient Descent
# train_step = tf.train.AdamOptimizer().minimize(cross_entropy) # Adam


sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
train_start = time.time()
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
train_end = time.time()
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('\nTest Accuracy: ' + str(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})))


tf_end = time.time()
end = time.time()
print('Data Preprocessing Elapsed Time: ' + str(data_end-data_start))
print('TensorFlow         Elapsed Time: ' + str(tf_end-tf_start))
print('Training           Elapsed Time: ' + str(train_end-train_start))
print('Total              Elapsed Time: ' + str(end-start))

Extracting ./MNIST-DATA/train-images-idx3-ubyte.gz
Extracting ./MNIST-DATA/train-labels-idx1-ubyte.gz
Extracting ./MNIST-DATA/t10k-images-idx3-ubyte.gz
Extracting ./MNIST-DATA/t10k-labels-idx1-ubyte.gz

Test Accuracy: 0.8761
Data Preprocessing Elapsed Time: 0.5083792209625244
TensorFlow         Elapsed Time: 1.0171470642089844
Training           Elapsed Time: 0.8346719741821289
Total              Elapsed Time: 1.5256309509277344
